In [1]:
import rank_mod
import build
import argparse
import sys
import iohelp
import variant
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from pysam import FastaFile
import resource
import time
from util import *

In [2]:
varstest = '../test/sm.1ksnp'
referencetest = '../test/sm.fa'
chromtest = '22'

In [3]:
method = 'popcov-blowup'
reference = referencetest #'../../hs37d5.fa'
vars = varstest #'../../chr6.1ksnp'
window_size = 100
output1 = '../../'+method+'-mod.txt'
chrom = chromtest#'6'
rankpath = output1
pct = 10
output2 = '../../build-mod-'+str(pct/100)+'.txt'

In [4]:
parser = argparse.ArgumentParser(description=__doc__,
            formatter_class=argparse.RawDescriptionHelpFormatter)
parser.add_argument('--method', default=method)
parser.add_argument('--reference', default=reference)
parser.add_argument('--vars', default=vars)
parser.add_argument('--window-size', default=window_size)
parser.add_argument('--chrom', default=chrom)
parser.add_argument('--output', default=output1)

parser.add_argument('--prune', type=int, required=False)
parser.add_argument('--phasing', type=str, required=False)
parser.add_argument('--pseudocontigs', action="store_true")

parser.add_argument('--hisat', type=str, required=False, help='Path to file to write HISAT2 --snp information')
parser.add_argument('--erg', type=str, required=False, help='Path to fasta file to write additional pseudocontigs for ERG alignment')
parser.add_argument('--sorted', type=str, default=rankpath)
parser.add_argument('--pct', type=int, required=False, help='Percentage of variants to include in graph genome')


args = parser.parse_args("")

In [5]:
parser1 = argparse.ArgumentParser(description=__doc__,
            formatter_class=argparse.RawDescriptionHelpFormatter)
parser1.add_argument('--method', default=method)
parser1.add_argument('--reference', default=reference)
parser1.add_argument('--vars', default=vars)
parser1.add_argument('--window-size', default=window_size)
parser1.add_argument('--chrom', default=chrom)
parser1.add_argument('--output', default=output2)

parser1.add_argument('--prune', type=int, required=False)
parser1.add_argument('--phasing', type=str, required=False)
parser1.add_argument('--pseudocontigs', action="store_true")

parser1.add_argument('--hisat', type=str, required=False, help='Path to file to write HISAT2 --snp information')
parser1.add_argument('--erg', type=str, required=False, help='Path to fasta file to write additional pseudocontigs for ERG alignment')
parser1.add_argument('--sorted', type=str, default=rankpath)
parser1.add_argument('--pct', type=int, default=pct, required=False, help='Percentage of variants to include in graph genome')


args1 = parser1.parse_args("")

In [6]:
def go(args):
    start_time = time.time()
    x1 = 'Memory usage start: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    if args.window_size:
        r = args.window_size
    else:
        r = 35
    if args.output:
        o = args.output
    else:
        o = 'ordered.txt'
    if args.prune:
        max_v = args.prune
    else:
        max_v = r
    genome1 = {args.chrom: ''}
    if args.method == 'hybrid':
        genome1 = iohelp.read_genome1(args.reference, args.chrom)
    x2 = 'Memory usage after  genome created: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    vars1 = iohelp.parse_1ksnp1(args.vars)
    x3 = 'Memory usage after  variants created: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    ranker = VarRanker1(genome1, vars1, r, args.phasing, max_v)
    ranked = ranker.rank(args.method, o)
    x4 = 'Memory usage after  ranking created: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    x5 = "Running time: %s seconds" % (time.time() - start_time)
    oram = 'ram-'+args.method+'-mod.txt'
    with open(oram, 'w') as f:
        f.writelines([x1,'\n',x2,'\n',x3,'\n',x4,'\n',x5])

In [7]:
class VarRanker1:
    def __init__(self, genome, variants, r, phasing, max_v):
        self.genome = genome
        self.chrom_lens = dict()
        for chrom, seq in genome.items():
            self.chrom_lens[chrom] = len(seq)

        self.variants = variants
        self.num_v = self.variants.shape[0]
        self.r = r

        self.phasing = phasing
        self.hap_parser = iohelp.HaplotypeParser(phasing) if phasing else None

        self.max_v_in_window = max_v

        self.h_ref = None
        self.h_added = None

        self.wgt_ref = None
        self.wgt_added = None

        self.curr_vars = None
        self.freqs = {}

    def rank(self, method, out_file):
        ordered = None
        ordered_blowup = None
        print(method)
        if method == 'popcov':
            ordered = self.rank_pop_cov()
        elif method == 'popcov-blowup':
            ordered = self.rank_pop_cov(True)
        elif method == 'hybrid':
            ordered, ordered_blowup = self.rank_hybrid()

        if ordered is not None:
            df = pd.DataFrame(columns = ['a', 'b'])
            df['a'] = self.variants.loc[ordered].chrom
            df['b'] = self.variants.loc[ordered].pos +1
            df.to_csv(out_file,header=False,index=False,line_terminator = '\t')
            # with open(out_file, 'w') as f:
            #     f.write('\t'.join([str(self.variants.chrom[i]) + ',' for i in ordered]))
        if ordered_blowup:
            df = pd.DataFrame(columns = ['a', 'b'])
            df['a'] = self.variants.loc[ordered_blowup].chrom
            df['b'] = self.variants.loc[ordered_blowup].pos +1
            df.to_csv(out_file,header=False,index=False,line_terminator = '\t')
            # with open(out_file+'.blowup', 'w') as f:
            #     f.write('\t'.join([self.variants.chrom[i] + ',' + str(self.variants.pos[i]+1) for i in ordered_blowup]))

    def rank_pop_cov(self, with_blowup=False, threshold=1/3):
        '''
        Rank variants using the hybrid ranking method.
        
        with_blowup: If true, add blowup penalty for neighboring variants
        threshold: Blowup penalty for neighboring variants, between 0 and 1. A smaller value is a stricter penalty
        '''
        
        if with_blowup:
            varsdf = self.variants
            var_wgts = (self.variants.iloc[:,[4,8,9]].sum(axis = 1)).sort_index()
            posdf = varsdf.pos
            ffirst = np.vectorize(
                lambda i: 
                    (posdf <= posdf[i]-r).idxmin()
                    )

            flast = np.vectorize(
                lambda i: 
                    (posdf >= posdf[i]+r).idxmax()
                    )

            first = np.maximum(0,np.fromfunction(ffirst,(self.num_v,), dtype=int))
            last = np.minimum(self.num_v,np.fromfunction(flast,(self.num_v,), dtype=int))-1
            last[last == -1] = self.num_v-1
            neighbors = last-first
            penalty = np.repeat(threshold,self.num_v)
            var_wgts = pd.Series(var_wgts* (penalty**neighbors).copy()).sort_values(ascending=False)
            print(var_wgts)
            ordered = var_wgts.index
        else:
            # Variant weight is the sum of frequencies of alternate alleles
            var_wgts = (self.variants.iloc[:,[4,8,9]].sum(axis = 1)).sort_index()
            var_wgts = var_wgts.sort_values(ascending=False)
            ordered = var_wgts.index

        return ordered

In [8]:
def read_genome1(filename, target_chrom=None):
    rec = FastaFile(filename)
    lengths = {rec.references[i]:rec.lengths[i] for i in range(len(rec.lengths))}
    return( {target_chrom:rec.fetch(reference=target_chrom, start = 0, end = lengths[target_chrom])})

In [9]:
def parse_1ksnp1(filename, G=None):
    df = pd.read_csv(filename, sep = "\t",header=None)
    df.columns = ["chrom", "pos", "orig", "alts", "probs1", 'x', 'num', "name"]
    df.pos = df.pos-1
    df['probs2'] = np.zeros((df.shape[0]))
    df['probs3'] = np.zeros((df.shape[0]))
    rem = []
    for num in range(1,4):
        indices = pd.Series(df.index[df.num == num])
        s = indices.iloc[np.arange(len(indices)//num)*num]
        for j in range(1,num):
            col = 'probs'+str(j+1)
            df[col].iloc[s] = np.array(df['probs1'].iloc[s+j])
            df['alts'].iloc[s] += np.array(df['alts'].iloc[s+j])
            rem.append(s+j)
    df = df.drop(pd.concat(rem).sort_values()).reset_index(drop=True)
    return df

In [10]:
rank_mod.go(args)
print("asdas")
build.go(args1)

popcov-blowup
asdas
genome check
variants check
[('22', 11190), ('22', 161571), ('22', 8010), ('22', 294874), ('22', 102546), ('22', 402330), ('22', 93886), ('22', 109000), ('22', 295341), ('22', 100529)]
Found 41 / 41 variants


/home/martin/Documents/VSCode/Proyecto BCOM4006/FORGe-mod/src/iohelp.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col].iloc[s] = np.array(df['probs1'].iloc[s+j])
/home/martin/Documents/VSCode/Proyecto BCOM4006/FORGe-mod/src/iohelp.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['alts'].iloc[s] += np.array(df['alts'].iloc[s+j])
